## Preparation

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
import sys
import time
import argparse
import numpy as np
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda:0') 

parser = argparse.ArgumentParser(description='dense')
parser.add_argument('--num_epochs', type=int, default=1)
parser.add_argument('--log_every', type=int, default=400)
parser.add_argument('--batch_size', type=int, default=4096)
parser.add_argument('--lr', type=float, default=0.0001)
args = parser.parse_args(args=[])



## Model

In [33]:
torch.cuda.current_device()
torch.cuda.get_device_name(0)

'Quadro P1000'

In [34]:
class RealDataset(Dataset):
  
    def __init__(self, x_train, y_train):
       
        self.x_train = x_train
        self.y_train = y_train


    def __len__(self):
        return len(self.x_train)

    def __getitem__(self, idx):
        x = self.x_train[idx]
       # x = torch.tensor(x, dtype = torch.float32)
        y = self.y_train[idx]
        y=torch.tensor(y, dtype = torch.long)
        return x, y
    

In [35]:
class LongDense(nn.Module):
    def __init__(self):
        super(LongDense, self).__init__()
        
        self.fc = nn.Sequential(
            nn.Linear(108, 752),
            nn.ReLU6(),
            nn.Linear(752, 576),
            nn.ReLU6(),
            nn.Linear(576, 256),
            nn.ReLU6(),
            nn.Linear(256, 192),
            nn.ReLU6(),
           # nn.Dropout(p=0.2),
            nn.Linear(192, 96),
            nn.ReLU6(),
            nn.Linear(96, 3),
            nn.Sigmoid()
        )


    def forward(self, x):
        x = self.fc(x)
        return x



## Load data

In [5]:
fid = pd.HDFStore("all_rad_train_test_data_B_log_R_cpuBG_H.h5")
x_train = fid['x_train']
x_test = fid['x_test']
y_train = fid['y_train']
y_test = fid['y_test']

In [6]:
x_train=x_train.values
y_train=y_train.values
x_train=x_train.astype(np.float32)
y_train=y_train.astype(np.int)
x_test=x_test.values
y_test=y_test.values
x_test=x_test.astype(np.float32)
y_test=y_test.astype(np.int)


In [36]:
traindataset = RealDataset(x_train=x_train, y_train=y_train)
train_loader = torch.utils.data.DataLoader(traindataset,
        batch_size=args.batch_size, shuffle=True)
testdataset = RealDataset(x_train=x_test, y_train=y_test)
test_loader = torch.utils.data.DataLoader(testdataset,
        batch_size=args.batch_size, shuffle=True)

## Main function

In [39]:
longdense = LongDense()
longdense.cuda()
longdense.train()
optimizer = torch.optim.Adam(longdense.parameters(), lr=args.lr, 
            betas=(0.9, 0.999), eps=1e-7)
    
for i in range(args.num_epochs):
    startep=time.time()
    for j, (inputdata, labels) in enumerate(train_loader):
        start = time.time()
            
        inputdata = inputdata.cuda()
        labels = labels.cuda()

        optimizer.zero_grad()  
        pred = longdense(inputdata)
        loss = nn.CrossEntropyLoss()(pred, labels.view(-1))
        loss.backward()
        optimizer.step()    
        end = time.time()
        if j % args.log_every == 0:
            train_acc = torch.mean((torch.max(pred, 1)[1] == labels).type(torch.float))
            display = 'epoch=' + str(i) + \
                      '\tacc=%.4f' % (train_acc) + \
                      '\tloss=%.4f' % (loss) + \
                      '\ttime=%.2fit/s' % (1. / (end - start+0.0001))
            print(display)
    endep=time.time()
    print('\ttime=%.2fit/s' % (1. / (endep - startep)))
        
longdense.eval()
total = 0.
acc_sum = 0.
for i, (inputdata, labels) in enumerate(test_loader):
    start = time.time()
            
    inputdata = inputdata.cuda()
    labels = labels.cuda()
    with torch.no_grad():
        pred = longdense(inputdata)
    acc_sum += torch.sum((torch.max(pred, 1)[1] == labels).type(torch.float))
    total += pred.shape[0]
    acc = acc_sum / total
    end = time.time()
    if (i) % args.log_every == 0:
        display = '\tacc=%.4f' % (acc) + \
                      '\ttime=%.2fit/s' % (1. / (end - start+0.0001))
        print(display)    

epoch=0	acc=0.6604	loss=1.0897	time=48.63it/s
epoch=0	acc=0.6572	loss=0.8551	time=54.24it/s
epoch=0	acc=0.6716	loss=0.8130	time=52.69it/s
epoch=0	acc=0.5382	loss=0.7432	time=52.79it/s


KeyboardInterrupt: 

In [16]:
a=np.sum(y_train==0)

In [17]:
b=np.sum(y_train==1)

In [18]:
c=np.sum(y_train==2)

In [19]:
a/(a+b+c)

0.6623140740305725

In [24]:
b/(a+b+c)

0.24088487647365406